In [ ]:
from esm.models.esmc import ESMC
from esm.sdk.api import ESMProtein, LogitsConfig
import torch
import torch.nn as nn
import pandas as pd
import torch
import numpy as np
from transformers import EsmTokenizer, EsmModel, Trainer, TrainingArguments
# from accelerate import Accelerator
from sklearn.model_selection import train_test_split
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.utils import clip_grad_norm_
from sklearn.preprocessing import StandardScaler
from torch.utils.data.dataloader import default_collate

: 

In [ ]:
def ESMC_func(sequence:str):
   '''调用ESMC模型进行embedding
   
   输入：蛋白质序列
   输出：embedding后的隐层特征（Size = [36, 1, 68, 1157]）
   '''
   protein = ESMProtein(sequence= sequence)
   # client = ESMC.from_pretrained("esmc_600m").to(device) # or "cpu"
   client = ESMC.from_pretrained("esmc_600m").to('cuda') # or "cpu"
   protein_tensor = client.encode(protein)
   logits_output = client.logits(protein_tensor, LogitsConfig(sequence=True, return_embeddings=True, return_hidden_states=True))
   return logits_output.hidden_states.to('cpu')

In [5]:
# df = pd.read_csv("/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_2.0/dataset/dataset/ESMC_Embedding_512/generated_EK_withpath_train_512.csv")
# df = df.loc[1,'ESMC_Embedding_Path']
tensor = torch.load('/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_2.0/dataset/dataset/ESMC_Embedding_512/merge/embedding_3.pt')
print(tensor.shape)

/tmp/ipykernel_1871554/558668244.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load('/home/users/hcdai/AI-peptide/Seq2Score/Seq2Score_2.0/dataset/datase

torch.Size([36, 1, 512, 1152])


In [2]:
ESMC_model = ESMC.from_pretrained('esmc_600m').to('cuda')




In [ ]:
def ESMC_encoding(sequence:str):
    '''
    调用ESMC进行Encoding，但是不进行计算，用途是输出forward方法可以接受的张量
    '''
    protein = ESMProtein(sequence = sequence)
    protein_tensor = ESMC_model.encode(protein)
    return protein_tensor

a = ESMC_encoding({'|!@#','asdf'}).to('cuda')
# b = a.sequence



In [24]:
class testnet(nn.Module):
    def __init__(self,ESMC):
        super().__init__()
        self.ESMC = ESMC
    def forward(self, x):
        x = self.ESMC.logits(x,LogitsConfig(sequence = True,return_hidden_states = True))
        return x
    
model = testnet(ESMC_model).to('cuda')
# x = b.to('cuda')
output = model(a).hidden_states[-1, :, :, :].shape
output = output
print(output)

torch.Size([1, 6, 1152])


In [ ]:
from esm.tokenization import get_esmc_model_tokenizers

def ESMC_600M_202412(device: torch.device | str = "cpu", use_flash_attn: bool = True, model_path: str = None):
    with torch.device(device):
        model = ESMC(
            d_model=1152,
            n_heads=18,
            n_layers=36,
            tokenizer=get_esmc_model_tokenizers(),
            use_flash_attn=use_flash_attn,
        ).eval()
    if model_path is not None:
        state_dict = torch.load(
            model_path,
            map_location=device,
            weights_only=True,
        )
        model.load_state_dict(state_dict)
    else:
        print(f"Error: The model file at {model_path} was not found.")

    return model

model = ESMC_600M_202412(device = 'cuda', use_flash_attn= True,model_path = "/home/users/hcdai/AI-peptide/Seq2Score/Seq2Rosscore/ESMC-MLP/ESMC-600M/data/wetights/esmc_600m_2024_12_v0.pth")

Error: The model file at None was not found.


In [1]:
input_df = '/home/users/hcdai/AI-peptide/Seq2Score/Henya/dataset/Example_majority.csv'
input_df = pd.read_csv(input_df)

NameError: name 'pd' is not defined

In [ ]:
seq1 = 'ASDF'
seq2 = '!@#|'
a = ESMC_encoding(seq1).to('cuda')
b = ESMC_encoding(seq2).to('cuda')
list_test = [a,b]

